In [ ]:
################################################Task 1############################################################

In [ ]:
#import of parquet file
import pandas as pd   
parquet_file = '/Users/juliagunevic/Documents/PRACA/yellow_tripdata_2022-01.parquet' 
df=pd.read_parquet(parquet_file, engine='pyarrow')
df

In [ ]:
#setting tpep droppoff datetime as index
df = df.set_index(df['tpep_dropoff_datetime'])
df = df.sort_index()

In [ ]:
#splitting parquet file into csv files
from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2022, 1, 1)
end_date = date(2022, 1, 31)
for single_date in daterange(start_date, end_date):
    single_date_edit=single_date.strftime("%m-%d-%Y")
    df_date = df[single_date_edit]
    df_date.to_csv(r'/Users/juliagunevic/Documents/PRACA/'+single_date_edit+'.csv', index = False)

In [ ]:
################################################Task 2############################################################

In [ ]:
#importing needed libraries
import os
import numpy as np
import psycopg2

In [ ]:
#importing one csv file as example
df2=pd.read_csv("01-01-2022.csv")
df2.head()

In [ ]:
#clean table names
file="01-01-2022"
clean_tbl_name=file.lower().replace("-","_") 
clean_tbl_name

In [ ]:
#clean column names
df2.columns=[x.lower() for x in df2.columns]
df2.columns

In [ ]:
 #Creation of example table
create table yellow_taxi_01_01_2022
    (vendorid int,
     tpep_pickup_datetime varchar,
     tpep_dropoff_datetime varchar,
     passenger_count float,
     trip_distance float,
     ratecodeid int,
     store_and_fwd_flag varchar,
      pulocationid int,
     dolocationid int,
     payment_type int,
     fare_amount float,
     extra float,
    mta_tax float,
     tip_amount float,
     tolls_amount float,
     improvement_surcharge float,
     total_amount float,
     congestion_surcharge float,
     airport_fee float,
    ); 

In [ ]:
replacements={
    'object':'varchar',
    'float64':'float',
    'int64': 'int',
    'datetime64':'timestamp',
    'timedelta64[ns]':'varchar'
}
replacements

In [ ]:
col_str=", ".join("{} {}".format(n,d) for (n,d) in zip(df2.columns, df2.dtypes.replace (replacements)))
col_str

In [ ]:
#NOT WORKING!!!
#security group changed to all trafic
#region as well changed, still not working

#open database connection
conn_string ="host='database-1.chkvqbpuyll8.eu-central-1.rds.amazonaws.com' dbname='database-1' user='juliagunevic' password='password.'"
conn=psycopg2.connect(conn_string)
cursor=conn.cursor()
print('opened database successfully')

In [ ]:
#drop table with the same name
cursor.execute("drop table if exists yellow_taxi_01_01_2022")


In [ ]:
#creation of tables in db
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2022, 1, 1)
end_date = date(2022, 1, 31)
for single_date in daterange(start_date, end_date):
    cursor.execute("create table yellow_taxi"+single_date+"\
(vendorid int, tpep_pickup_datetime varchar, tpep_dropoff_datetime varchar, passenger_count float,trip_distance float, ratecodeid float, store_and_fwd_flag varchar, pulocationid int, dolocationid int, payment_type int, fare_amount float, extra float, mta_tax float, tip_amount float, tolls_amount float, improvement_surcharge float, total_amount float, congestion_surcharge float, airport_fee float)")
    #insert values to table
    my_file=open(single_date+'.csv')
    #upload to db
    SQL_STATEMENT="""

    COPY"""+single_date+""" FROM STDIN WITH
    CSV
    HEADER
    DELIMITER AS ","

    """
    cursor.copy_expert(sql=SQL_STATEMENT, file=my_file)
print("file copied to db")

In [ ]:
################################################Task 3############################################################

In [ ]:
print (df.describe())

In [ ]:
 #testing completness of data
percent_missing = df.isnull().sum() * 100 / len(df)
percent_missing

In [ ]:
#deleting rows with missing data
df=df.dropna()

In [ ]:
#testing conformity
df.dtypes

In [ ]:
#testing uniqueness 
df.nunique()

In [ ]:
################################################Task 4############################################################

In [ ]:
# I did not understand how to create star schema with just one table. I could do one, if there would be few tables
# and to draw relationship between them
#to my best knowledge I did example of star schema which you can find on github

In [ ]:
################################################Task 5############################################################

In [ ]:
# I believe I could do better the upload of data to databse. It is a pity that it doesn't work.
#what is more, there is room to grow in the section of data quality.I feel I could not do it to my best knowledge
#as I lacked information about what for example values should be put in specific columns. Because of that I could
#not do the validation part of data quality.